In [1]:
import re
import sys; sys.path.append("../")

import pandas as pd

from lib.columns import rearrange_personel_columns
from lib.textract import realign_series_ws, realign_series_val, parse_textract_datetime

In [2]:
df = pd.read_csv("../data/Gretna Police Department/Administrative Data/Gretna_PD_PPRR_2018.csv")
df.rename(columns=lambda x: x.strip(), inplace=True)
df.loc[:, "Rank"] = realign_series_val(df["Rank"], ["Sergeant Officer"])
df.loc[:, "Hire Date"] = realign_series_ws(df["Hire Date"])
df.loc[:, "2018 SALARY"] = realign_series_ws(df["2018 SALARY"])
df

,Name,Rank,Hire Date,2018 SALARY
0,"CHRISTIANA, ANTHONY",Deputy Chief (d.),4/11/83,"$100,754.61"
1,"LLOYD, RUSSELL",Captain,4/11/83,"$88,844.60"
2,"JACKSON, JARVIS",Captain,1/30/85,"$77,144.08"
3,COVELL. ERIC,Captain,10/11/86,"$40,163.37"
4,"DEBLANC, LILA",Officer,4/20/87,"$22,632.79"
...,...,...,...,...
111,"PITRE, NICHOLAS",Officer,1/26/18,"$19,071.06"
112,"CASH, MICHAEL",Officer,5/3/18,"$24,579.89"
113,"ELLIS, DANNY",Officer,5/3/18,"$21,228.80"
114,"HENDERSON, DOUGLAS",Officer,5/3/18,"$9,934.59"


In [3]:
df2 = df[["Rank", "Hire Date", "2018 SALARY"]]
df2.rename(columns={
    "2018 SALARY": "Annual Salary",
}, inplace=True)
df2["Hire Date"] = parse_textract_datetime(df2["Hire Date"])
df2.loc[:, "Annual Salary"] = df2["Annual Salary"].str.replace("$", "", regex=False).str.replace(",", "", regex=False).str.strip().astype("float64")
names = df["Name"].str.title().str.replace(". ", ", ", regex=False).str.split(", ", expand=True)
df2.loc[:, "First Name"] = names.iloc[:, 1]
df2.loc[:, "Last Name"] = names.iloc[:, 0].str.replace("(.+)(Ii|Iii|Iv)$", lambda m: m.group(1) + m.group(2).upper())
df2 = rearrange_personel_columns(df2)
df2


,Last Name,First Name,Rank,Hire Date,Annual Salary
0,Christiana,Anthony,Deputy Chief (d.),1983-04-11,100754.61
1,Lloyd,Russell,Captain,1983-04-11,88844.60
2,Jackson,Jarvis,Captain,1985-01-30,77144.08
3,Covell,Eric,Captain,1986-10-11,40163.37
4,Deblanc,Lila,Officer,1987-04-20,22632.79
...,...,...,...,...,...
111,Pitre,Nicholas,Officer,2018-01-26,19071.06
112,Cash,Michael,Officer,2018-05-03,24579.89
113,Ellis,Danny,Officer,2018-05-03,21228.80
114,Henderson,Douglas,Officer,2018-05-03,9934.59


In [4]:
df2.to_csv("../output/gretna-pd_personel_2018.csv", index=False)